# ch 12. Improving training with metrics and augmentation
---
이전 장에서 모델을 테스트해본 결과 99%의 높은 정확도를 보여줬다. 하지만 이는 데이터의 불균형으로 인해 믿을 수 없는 지표가 되버렸다.

실제 데이터의 분포는 악성 종양이 0.3% 정도만 있기 때문에 모델이 전부 악성 종양이 아니라고 답하면 높은 정확도가 나오기 때문이다.

이를 개선하기 위해 다양한 metric을 알아본다.

### 거짓 양성과 거짓 음성

 - **거짓 양성**(**False Positive**): 결절이 아닌데 결절로 감지되는 경우
 - **참 양성**(**True Positive**): 결절을 결절로 잘 분류한 경우
 - **거짓 음성**(**False Negative**): 결절을 결절로 분류하지 못한 경우
 - **참 음성**(**False Negative**): 결절이 아닌 것을 잘 분류한 경우

![image](https://user-images.githubusercontent.com/76675506/190993717-f5b8dd13-f174-4541-ac4a-d6f02696bc30.png)


### 재현율, 정밀도, F1 점수

- **재현율**(**Recall**):

![image](https://user-images.githubusercontent.com/76675506/191006476-a0e39322-0bd6-4763-882d-8dea70adb595.png)

개선하기 위해서는 거짓 음성을 줄이면 된다.

- **정밀도**(**Precision**):

![image](https://user-images.githubusercontent.com/76675506/190994818-7dc5672d-2cef-4ce4-8c2d-dfe1a001c7de.png)

개선하기 위해서는 거짓 양성을 낮춰야 한다.

- **F1 점수**(**F1 Score**)

![image](https://user-images.githubusercontent.com/76675506/191013314-9f14d30d-60da-49b5-846e-c38954a46b4d.png)

![image](https://user-images.githubusercontent.com/76675506/191013545-722d12ab-b62f-49b7-90e6-728d3bfa2281.png)


세 가지 지표를 코드로 나타내면 아래와 같다.

In [ ]:
        neg_count = int(negLabel_mask.sum())
        pos_count = int(posLabel_mask.sum())

        trueNeg_count = neg_correct = int((negLabel_mask & negPred_mask).sum())
        truePos_count = pos_correct = int((posLabel_mask & posPred_mask).sum())

        falsePos_count = neg_count - neg_correct
        falseNeg_count = pos_count - pos_correct

        precision = metrics_dict['pr/precision'] = \
            truePos_count / np.float32(truePos_count + falsePos_count)
        recall    = metrics_dict['pr/recall'] = \
            truePos_count / np.float32(truePos_count + falseNeg_count)

        metrics_dict['pr/f1_score'] = \
            2 * (precision * recall) / (precision + recall)

위 지표를 이용해 모델을 평가해본 결과 아래 사진과 같이 나타났다.

정확도와 재현율이 모두 0으로 나타났고 이 때문에 F1은 nan값으로 나오고 있다.

![image](https://user-images.githubusercontent.com/76675506/191015889-35f7aad0-06c4-4451-b714-91cee77598d6.png)

## 불균형한 데이터

위와 같은 결과가 나온 이유는 불균형한 분포의 데이터 때문이다.